# 数值稳定性

## 一、比喻

先用一个生活例子理解  
想象你在玩传话游戏：  
第1个人 → 第2个人 → … → 第100个人  
每个人传话时，要么会"夸大"一点，要么会"缩小"一点  

- 每个人都稍微夸大一点（×1.5）：传100次后，原话被放大到离谱 → **梯度爆炸**  
- 每个人都稍微缩小一点（×0.8）：传100次后，原话几乎听不见了 → **梯度消失**

神经网络的梯度反向传播就是这个传话过程！

### 核心原理

神经网络的梯度 = 一堆矩阵连乘  
网络有d层：  
梯度 = ∂L/∂h_d × ∂h_d/∂h_(d-1) × ... × ∂h_(t+1)/∂h_t × ∂h_t/∂w_t  
(d-t) 个矩阵相乘

问题就出在这里：你要把很多矩阵乘在一起  

比如：  

- 1.5 × 1.5 × 1.5 × ... （100次）= 4 × 10^17 → **爆炸**  
- 0.8 × 0.8 × 0.8 × ... （100次）= 2 × 10^-10 → **消失**

## 二、梯度爆炸与梯度消失详解

### 梯度爆炸 —— 数越乘越大

**什么时候会发生？**  
当每层梯度的值 > 1 时，层数一多，连乘就会导致数值急剧增大，最终引发梯度爆炸。

**以 ReLU 激活函数为例：**  

- ReLU 的导数 = **1**（输入 > 0 时）或 **0**（输入 ≤ 0 时）  
- 反向传播中的梯度 ≈ $ W_1 \times W_2 \times W_3 \times \cdots $ 的部分元素相乘  
- 如果权重矩阵中的值普遍略大于 1 → 连乘后结果呈指数级增长 → **爆炸**

**会带来什么问题？**  

- **问题1：数值变成 inf（无穷大）**  
  → 尤其在使用 **16位浮点数**（FP16）时，数值范围仅为 -65504 到 +65504  
  → 一旦超出该范围，数值变为 `inf`，程序直接崩溃  

- **问题2：学习率极难调节**  
  → 学习率稍大 → 参数更新幅度过大 → 梯度进一步放大 → 引发爆炸  
  → 学习率过小 → 参数几乎不更新 → 训练停滞  
  → 只有非常狭窄的学习率区间可用，调参极其困难

### 梯度消失 —— 数越乘越小

**什么时候会发生？**  
当每层梯度的值 < 1 时，随着网络层数增加，多个小于1的数连续相乘，结果趋近于零，造成梯度消失。

**以 Sigmoid 激活函数为例：**  

- Sigmoid 函数：$ \sigma(x) = \frac{1}{1 + e^{-x}} $  
- 其导数：$ \sigma'(x) = \sigma(x)(1 - \sigma(x)) $

| 特性 | 说明 |
| --- | --- |
| **Sigmoid 导数的最大值** | **只有 0.25** |
| **输入稍大（如 > 4）时** | 导数接近 0 |

因此，每一层的梯度都小于 0.25，经过多层连乘：
$$
0. 25 \times 0.25 \times 0.25 \times \cdots \text{（100次）} \approx 0
$$

**会带来什么问题？**  

- **底层（靠近输入的层）根本学不动！**  
  反向传播是从输出层向输入层传递梯度的过程：

| 层级位置 | 梯度状态 | 学习能力 |
| --- | --- | --- |
| 顶层（靠近输出） | 梯度正常 ✅ | 正常更新参数 |
| 中间层 | 梯度开始变小 ⚠️ | 更新缓慢 |
| 底层（靠近输入） | 梯度 ≈ 0 ❌ | 完全无法学习 |

**结果：**  
你以为你搭建了一个100层的深度神经网络，  
但实际上只有顶部几层在学习，  
底层参数几乎不变，等同于一个浅层网络 —— **白搭了！**

### 一张图总结

| 梯度爆炸 | 梯度消失 |
| --- | --- |
| 原因：每层梯度 > 1，连乘后 → 越乘越大 | 原因：每层梯度 < 1，连乘后 → 越乘越小 |
| 激活函数举例：ReLU（当权重 > 1 时） | 激活函数举例：Sigmoid（导数最大仅 0.25） |
| 后果：数值溢出为 inf，程序崩溃 | 后果：底层梯度为 0，无法更新参数 |
| 学习率极难调 | 即使增大学习率也无济于事 |
| 网络越深越严重 | 网络越深越严重 |

### 你需要记住的

#### 🔴 必须记住

| 要点 | 内容 |
| --- | --- |
| **核心概念** | 梯度爆炸和梯度消失是深度网络训练的两大障碍 |
| **根本原因** | 反向传播本质是多个矩阵连乘，导致数值要么不断放大，要么不断缩小 |
| **Sigmoid 的坑** | 导数最大只有 0.25，深层网络中必然导致梯度消失，现已基本被淘汰 |
| **ReLU 为什么好** | 导数为 1 或 0，在正区域能保持梯度不衰减，有效缓解梯度消失 |
| **实战指导** | 优先选用 ReLU 类激活函数，权重需合理初始化（如 Xavier、He 初始化） |

-----------------

